In [2]:
"""
    Use the CNN model in order to make predictions on files in data_png_cnn2/all_png_test
    from: https://keras.io/examples/vision/image_classification_from_scratch/
"""

import sys
import datetime
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from keras.preprocessing import image
from keras.models import load_model

# add parent folder to path 
from pathlib import Path
path = Path(os.getcwd())
str_parent = str(path.parent.absolute())
if not str_parent in sys.path:
    sys.path.append(str_parent)
import utils 



ModuleNotFoundError: No module named 'keras'

In [ ]:
now = datetime.datetime.now()

image_size = (333, 216)
png_test_folder = '../../_data_png_cnn2/all_png_test/'
model_folder = '../../_classifiers/global_cnn.h5'

# load the model from disk
model = load_model(model_folder)

df_result = pd.DataFrame(columns=['file', 'score']) # df result  with files and scores
indiceFile = 0
nbPredictionOK = 0


In [ ]:
# normal and scorePredict of the class > thresholdPrediction -> accuracy ok
# normal and scorePredict of the class <= thresholdPrediction -> accuracy nok

# anomaly and scorePredict of the class > thresholdPrediction -> accuracy nok
# anomaly and scorePredict of the class <= thresholdPrediction -> accuracy ok
# anomaly if it doesn't ptedict the correct machine or the score < thresholdPrediction
thresholdPrediction = 0.95 
# thresholdPredictionMin = 0.10 

dict_machine = {'ToyCar': {'err': 0, 'total': 0}, 'ToyConveyor': {'err': 0, 'total': 0}, 'slider': {'err': 0, 'total': 0}, 'pump': {'err': 0, 'total': 0}, 'fan': {'err': 0, 'total': 0}, 'valve': {'err': 0, 'total': 0}} # stats accuracy by machine
list_machine = ['ToyCar', 'ToyConveyor', 'fan', 'pump', 'slider', 'valve'] # ok 
# browse test files
wavfiles = [f for f in listdir(png_test_folder) if isfile(join(png_test_folder, f))] # 7730 pngs
nbWavs = len(wavfiles)
for nameFilePngTotest in wavfiles:
    if nameFilePngTotest[-4:] != '.png': # ignore non .png files 
        continue
    arrName = nameFilePngTotest.split("_") # normal_id_06_00000451_pump.png
    classPrefixReal = arrName[0] # 'normal' or 'anomaly'
    classNameReal = arrName[4][:-4] # 'pump' / [:-4] remove the '.png'
    # print(className)
    if classPrefixReal != 'anomaly': # anomaly only  accuracy:  0.615
    # if classPrefixReal != 'normal': # normal only  accuracy:  0.568
        continue
    # print('test nameFilePngTotest: ', nameFilePngTotest)
    test_image = image.load_img(png_test_folder + nameFilePngTotest, target_size = image_size) # , target_size = (64, 64)
    img_array = image.img_to_array(test_image)
    img_array = np.expand_dims(test_image, axis = 0)
    arr_predictions = model.predict(img_array)
    arr_predictions = arr_predictions[0] # prediction one by one so use the [0]
    # print(nameFilePngTotest, np.rint(arr_predictions)) # [[0. 0. 0. 1. 0. 0.]]
    # print(nameFilePngTotest, np.round(arr_predictions, 2)) # [[0.   0.03 0.   0.95 0.03 0.  ]] 
    # print(nameFilePngTotest, np.rint(arr_predictions).argmax(axis = 1)) # 
    
    predictionOK = False
    indiceClassNamePredict = list_machine.index(classNameReal)
    scorePredict = arr_predictions[indiceClassNamePredict]
    if scorePredict > thresholdPrediction and classPrefixReal == 'normal' or scorePredict <= thresholdPrediction and classPrefixReal == 'anomaly' :
        predictionOK = True
        nbPredictionOK+= 1
    
    dict_machine[classNameReal]['total'] =  dict_machine[classNameReal]['total'] + 1
    # count errors by machine 
    if predictionOK == False:
        dict_machine[classNameReal]['err'] =  dict_machine[classNameReal]['err'] + 1
    # indiceClassNamePredict = np.rint(arr_predictions).argmax(axis = 1)[0]
    # if the prediction is right, sound is normal otherwise, it's anormal
    # classOK = True if classNameReal == list_machine[indiceClassNamePredict] else False
    # # if classNameReal == list_machine[indiceClassNamePredict]:
    # #     classOK = True
    # if classOK and classPrefixReal == 'normal' or not classOK and classPrefixReal == 'anomaly':
    #     predictionOK = True
    #     nbPredictionOK+= 1
    # print(nameFilePngTotest, result) # [[0. 0. 0. 1. 0. 0.]]
    # score = arr_predictions[0]

    print(nameFilePngTotest, np.round(arr_predictions, 2), list_machine[indiceClassNamePredict], predictionOK) #ok
    # df_result = df_result.append({'file': nameFilePngTotest, 'score': score}, ignore_index=True)
    indiceFile += 1
    # if indiceFile % 100 == 0:
    #     print('indiceFile: ', indiceFile, ' / ', nbWavs, ' nbPredictionOK: ', nbPredictionOK, ' accuracy: ', nbPredictionOK / indiceFile)
    if indiceFile == 200:
        break
# df_result = df_result.sort_values(by = ['score'], ascending = False)
# df_result.head(40)

accuracy = nbPredictionOK / indiceFile
print('indiceFile: ', indiceFile, ' / ', nbWavs, ' nbPredictionOK: ', nbPredictionOK, ' accuracy: ', accuracy)
print('dict_machine: ', dict_machine) 